In [0]:
import numpy as np 
import pandas as pd 
import os
import json
from tqdm import tqdm_notebook
import pickle

In [0]:
!pip install lightfm
from lightfm import LightFM

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
os.chdir('/content/drive/My Drive/Colab Notebooks Shared/IDA contests/Articles RecSys')

## Read data

In [0]:
items_list=[]
with tqdm_notebook(open('items.json')) as inf:
    for line in inf:
        item=json.loads(line)
        if isinstance(item['image'], float):
            item['image']=[0 for _ in range(96)]
            
        item['image']=np.array(item['image'])
        items_list.append(item)

items=pd.DataFrame(items_list)

In [0]:
train = read_json('train.json')
df = list()
for index, row in train.iterrows():
   for key, value in row['trainRatings'].items():
       df.append([row['userId'], key, value])
df = pd.DataFrame(df, columns = ['userId', 'itemId', 'react'])
del train

In [0]:
# df = pd.read_csv('train_user_item.csv', index_col=0)
test = pd.read_csv('random_benchmark.csv')

## Preprocessing

* Введем оценку просмотра статьи (за отсутствие взаимодействия 0, за просмотр ~0.9 - доля не просмотренных статей из всех показанных статей, за не просмотр -0.05 - небольшое отрицательное число).
* Зададим спарс матрицу всех вазимодействий пользователя и айтема.
* Обучим tfidf вектора на заголовках статей. Поскольку метки 0 и 1 отображают была ли прочитана статья, то имеет смысл смотреть только заголовок, т.к. содержание становится известно только тогда, когда статья уже прочитана. Параметры min_df и max_df подбирались так, чтобы размер вектора был максимально возможным из расчета, что эпоха обучается не дольше 10 минут.

In [0]:
df['react_score'] = (df['react'].replace(1, (df['react'] == 0).sum() / len(df['react']))).replace(0, -0.05)

In [0]:
from scipy.sparse import csr_matrix
interact_df = csr_matrix((df['react_score'], (df['userId'], df['itemId'])))

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(min_df=3, max_df=0.1)
tfidf_head = tfidf_vec.fit_transform(items['title'])
tfidf_head.shape

(328050, 72235)

In [0]:
test_gr = test.groupby('userId')['itemId'].agg(lambda it: list(it))

## Model

* В отправленном в anytask ноутбуке до этого были предствлены модели svd, NN, SGDClassifier и их комбинация, но их качество заметно проигрывает модели LightFM. Данная модель использует признаки айтемов, что позволяет решать проблему холодного старта (мы можем рекомендовать пользователям совсем новые айтемы), и использует продвинутые алгоритмы факторизации матриц (позволяет создавать векторное представление пользователя и айтема, чтобы оценивать их взаимодействие).

* Количество компонент подбиралось как гиперпараметр. Также если размер вектора достаточно большой, то имеет смысл брать большее количество компонент.

* При большом количестве эпох модель переобучается.

In [0]:
model = LightFM(no_components=100)

In [0]:
for i in tqdm_notebook(range(10)):
    model.fit_partial(interactions=interact_df, item_features=tfidf_head, epochs=1, num_threads=4)
    with open('LFM_100_head{0:02d}.pkl'.format(i), 'wb') as f:
        pickle.dump(model, f)

In [0]:
answ = list()
for uid, it in tqdm_notebook(test_gr.iteritems(), total=len(test_gr)):
    pred = model.predict(user_ids=uid, item_ids=np.array(it), item_features=tfidf_head)
    answ.extend(zip([uid]*len(pred), it, pred))
answ = pd.DataFrame(answ, columns=['userId', 'itemId', 'react'])
res = answ.sort_values(['userId', 'react'], ascending=[True, False])
res[['userId', 'itemId']].to_csv('LFM_100_head_answ10.csv', index=None)